<a href="https://colab.research.google.com/github/kayla-jackson/spatial-modeling/blob/test-exploratory-sims/notebooks/spatially_correlated_p.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
!pip install esda

In [71]:
import numpy as np
import matplotlib.pyplot as plt
from esda.moran import Moran

In [72]:
# Static variables

GRID_SIZE = 10 # dimensions of spatial grid
NUM_SAMPLES = 1000

In [73]:
# Helper functions to generate a sample from MVN(0, sigma)

def adjacency_matrix(A):
    '''Returns the adjacency matrix W from A, a NumPy array'''
    rows, cols = len(A), len(A[0])
    locs = rows * cols # number of spatial locations on the grid
    W = np.zeros((locs, locs), dtype=int)

    # Iterate through each element in A to determine number of neighbors
    for r in range(rows):
        for c in range(cols):
            for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
                if 0 <= r + dr < rows and 0 <= c + dc < cols:
                    W[r * cols + c][(r + dr) * cols + c + dc] = 1

    return W

def sum_adjacency_matrix(W):
    '''Outputs matrix D with same dimensions as W;
    Sums the number of neighbors of the original points'''
    return np.diag(np.sum(W, axis=1))

def compute_sigma(A, alpha, tau):
    '''Outputs the covariance matrix termed sigma from original matrix A'''
    W = adjacency_matrix(A)
    D = sum_adjacency_matrix(W)

    Q = tau * (D - alpha * W) # the formula
    return np.linalg.inv(Q)

In [74]:
# Testing the functions

A = np.array([[1,2,3],[4,5,6],[7,8,9]])
W = adjacency_matrix(A)
D = sum_adjacency_matrix(W)
sigma = compute_sigma(A, 0.99, 1)

print(f'W: {W}')
print(f'D: {D}')
print(f'Sigma: {sigma}')

W: [[0 1 0 1 0 0 0 0 0]
 [1 0 1 0 1 0 0 0 0]
 [0 1 0 0 0 1 0 0 0]
 [1 0 0 0 1 0 1 0 0]
 [0 1 0 1 0 1 0 1 0]
 [0 0 1 0 1 0 0 0 1]
 [0 0 0 1 0 0 0 1 0]
 [0 0 0 0 1 0 1 0 1]
 [0 0 0 0 0 1 0 1 0]]
D: [[2 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0]
 [0 0 2 0 0 0 0 0 0]
 [0 0 0 3 0 0 0 0 0]
 [0 0 0 0 4 0 0 0 0]
 [0 0 0 0 0 3 0 0 0]
 [0 0 0 0 0 0 2 0 0]
 [0 0 0 0 0 0 0 3 0]
 [0 0 0 0 0 0 0 0 2]]
Sigma: [[4.72557687 4.26825946 4.10427136 4.26825946 4.10427136 4.02319782
  4.10427136 4.02319782 3.98296585]
 [4.26825946 4.51847503 4.26825946 4.10427136 4.14572864 4.10427136
  4.02319782 4.02340102 4.02319782]
 [4.10427136 4.26825946 4.72557687 4.02319782 4.10427136 4.26825946
  3.98296585 4.02319782 4.10427136]
 [4.26825946 4.10427136 4.02319782 4.51847503 4.14572864 4.02340102
  4.26825946 4.10427136 4.02319782]
 [4.10427136 4.14572864 4.10427136 4.14572864 4.35427136 4.14572864
  4.10427136 4.14572864 4.10427136]
 [4.02319782 4.10427136 4.26825946 4.02340102 4.14572864 4.51847503
  4.02319782 4.1042

In [75]:
# With the full 10x10 grid:

A = np.zeros((GRID_SIZE, GRID_SIZE), dtype=int)
W = adjacency_matrix(A)
D = sum_adjacency_matrix(W)
sigma = compute_sigma(A, 0.99, 1)

print(f'W: {W}')
print(f'D: {D}')
print(f'Sigma: {sigma}')

W: [[0 1 0 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 1 0 1]
 [0 0 0 ... 0 1 0]]
D: [[2 0 0 ... 0 0 0]
 [0 3 0 ... 0 0 0]
 [0 0 3 ... 0 0 0]
 ...
 [0 0 0 ... 3 0 0]
 [0 0 0 ... 0 3 0]
 [0 0 0 ... 0 0 2]]
Sigma: [[1.44637106 0.95593036 0.68112832 ... 0.14572908 0.1416529  0.14023637]
 [0.95593036 1.16191315 0.77788798 ... 0.14658889 0.14288171 0.1416529 ]
 [0.68112832 0.77788798 1.04330611 ... 0.14958705 0.14658889 0.14572908]
 ...
 [0.14572908 0.14658889 0.14958705 ... 1.04330611 0.77788798 0.68112832]
 [0.1416529  0.14288171 0.14658889 ... 0.77788798 1.16191315 0.95593036]
 [0.14023637 0.1416529  0.14572908 ... 0.68112832 0.95593036 1.44637106]]


In [76]:
# Generating the sample

def MVN_sample(sigma, num_samples):
    '''Returns a sample from a multivariate normal distribution
    where sigma is a covariance matrix'''
    zero_vector = np.zeros(len(sigma))
    samples = np.random.multivariate_normal(zero_vector, sigma, num_samples)

    return samples

def inverse_logit_transformation(M):
    '''Transforms a matrix of values into the range (0,1)'''
    return 1 / (1 + np.exp(-M))

In [77]:
# Testing the above functions

samples = MVN_sample(sigma, NUM_SAMPLES)
transformed_samples = inverse_logit_transformation(samples)
print(samples)
print(transformed_samples)

[[ 0.2520072   0.45280747  0.97306915 ...  1.04027384  1.01729803
   1.3497599 ]
 [ 0.47767184  0.07783905  0.37196995 ...  1.10345179  0.24120777
  -0.39994919]
 [ 0.84365299  1.37144436  1.19352523 ...  0.99313615  0.08536261
   0.21827274]
 ...
 [ 0.96265152  0.36680148 -0.7983687  ...  0.64422927  0.14247527
  -0.35006468]
 [-0.8580198  -0.55318178 -1.02609138 ... -0.2089635   0.37944446
  -0.48907107]
 [-0.94207807 -0.6950328  -1.91839959 ...  0.07486801 -0.82440495
   0.09725892]]
[[0.56267048 0.61130653 0.72573082 ... 0.73890284 0.73444595 0.79409037]
 [0.61719796 0.51944994 0.5919349  ... 0.75090631 0.56001126 0.40132455]
 [0.69923402 0.79761341 0.76737096 ... 0.72970693 0.5213277  0.55435256]
 ...
 [0.72365237 0.59068588 0.31037458 ... 0.65570887 0.53555869 0.41336674]
 [0.29775323 0.36512653 0.26384257 ... 0.44794839 0.59373911 0.38011242]
 [0.28048077 0.33291444 0.12804014 ... 0.51870826 0.30482941 0.52429558]]


In [78]:
# Computing Moran's I for the 1000 samples

def moran_I(samples, adj_matrix):
    '''Returns array of Moran's I values for each sample in samples;
    adj_matrix is spatial weights matrix'''

    moran_values = []

    # Compute Moran's I for each sample
    for sample in samples:
        print(sample)
        moran_I = Moran(sample, adj_matrix).I
        moran_values.append(moran_I)

    return moran_values

In [79]:
import scipy.sparse as sp
from libpysal.weights import WSP

# Convert the adjacency matrix to a scipy sparse matrix
sparse_W = sp.csr_matrix(W)

# Create a PySAL weights object from the sparse adjacency matrix
W = WSP(sparse_W, id_order=range(sparse_W.shape[0]))

moran_values = moran_I(transformed_samples, W)
print(moran_values)

[0.56267048 0.61130653 0.72573082 0.5997967  0.45056663 0.53836388
 0.50409016 0.59611203 0.37025545 0.66972394 0.69461785 0.83313902
 0.6967451  0.4424289  0.35224835 0.49793813 0.47718291 0.80087967
 0.46136014 0.56763019 0.80364352 0.65530642 0.47663433 0.36296795
 0.52231495 0.44149516 0.5876388  0.75601024 0.45163918 0.54741605
 0.40750151 0.18667349 0.4594305  0.46572226 0.38569293 0.46416818
 0.60369125 0.70599945 0.48284548 0.69246035 0.30858526 0.2279436
 0.51683882 0.5855208  0.43745931 0.77017223 0.72347432 0.47989986
 0.37348052 0.71717106 0.23027159 0.19039368 0.22435685 0.46829437
 0.57384468 0.80042611 0.42951756 0.29599734 0.45676152 0.59560372
 0.11459184 0.13696796 0.19099682 0.25815324 0.48486006 0.58618666
 0.39370971 0.30751999 0.601367   0.64966289 0.46191789 0.28789221
 0.14342046 0.44090392 0.65295735 0.64335304 0.50622952 0.52962071
 0.70878027 0.67041053 0.12543726 0.16669191 0.09899749 0.41756575
 0.66832298 0.67334826 0.72389542 0.74313658 0.68240638 0.67431

AttributeError: ignored